Contenido bajo licencia Creative Commons BY 4.0 y código bajo licencia MIT. © Juan Gómez y Nicolas Guarín-Zapata 2019. Este material es parte del curso Modelación Computacional en el programa de Ingeniería Civil de la Universidad EAFIT.

# Two-dimensional frame elements

## Introducción

En este Notebook se agrega un elemento adicional al programa de análisis estructural, iniciado yá con el problema simple de resortes uni-dimesionales y extendido también para considerar armaduras planas. En partícular ahora se adicionan elementos tipo viga para abordar problemas en los cuales los elementos funcionen principalmente a flexión. De otro lado, en una implementación independiente, es posible acoplar el comportamiento a carga axial de la cercha con el comportamiento a flexión de la viga. El modelo de viga que se discute en este notebook corresponde a un modelo de Euler-Bernoulli en el cual se desprecian las deformaciones por cortante y por lo tanto valido para elementos con secciones transversales de baja esbeltez.

**Al completar este notebook usted debería estar en la capacidad de:**

* Reconocer las modificaciones necesarias para convertir un programa fundamental de ensamblaje de resortes en uno para estructuras conformadas por vigas.

* Resolver problemas simples de estructuras conformadas por vigas sometidas a cargas puntuales.

### Ensamblaje de elementos tipo viga

Consideremos un elemento tipo viga en su sistema local de referencia como el mostrado en la figura. En el sistema local el elemento tiene 2 grados de libertad por nudo, correspondientes al desplazamiento transversal y una rotación con respecto a un eje perpendicular al plano de la imagen.

<center><img src="images/frame1.png" alt="files" style="width:400px"></center>

El vector de grados de libertad (o desplazamientos generalizados) del elemento es:

$$
u^T=\begin{bmatrix}v_1&\theta_1&v_2&\theta_2\end{bmatrix}
$$

mientras que el vector de fuerza (momentos y cortantes) esta dado por:

$$
f^T=\begin{bmatrix}f_1&m_1&f_2&m_2\end{bmatrix}
$$


En el sistema de referencia local la matriz de rigidez relacionando fuerzas con desplazamientos es:

$$
$$
\begin{bmatrix}12\frac{EI}{\mathcal l^3}&6\frac{EI}{\mathcal l^3}&-12\frac{EI}{\mathcal l^3}&6\frac{EI}{\mathcal l^2}\\6\frac{EI}{\mathcal l^3}&4\frac{EI}{\mathcal l}&-6\frac{EI}{\mathcal l^2}&2\frac{EI}{\mathcal l}\\12\frac{EI}{\mathcal l^3}&-6\frac{EI}{\mathcal l^2}&12\frac{EI}{\mathcal l^3}&-6\frac{EI}{\mathcal l^2}\\6\frac{EI}{\mathcal l^2}&2\frac{EI}{\mathcal l}&-6\frac{EI}{\mathcal l^2}&4\frac{EI}{\mathcal l}\end{bmatrix}
$$
$$

**Nota: La matriz de rigidez puede ser formulada por diferentes métodos que se cubren en el curso de Análisis de Estructuras**

Para obtener la matriz de rigidez global de la estructura es necesario considerar nuevamente la contribución de todos los elementos en el sistema **Global** de referencia. Con ese proposito procedemos como con el problema de la cercha. Usando la matriz de transfomración bajo rotación $\lambda$ se tiene que:

$$K=\lambda^Tk\lambda$$

donde $K$ es la matriz de rigidez para el elemento tipo viga en el sistema global de referencia. Note que en este sistema el elemento tiene ahora 3 grados de libertad por nudo.

### Estructura aporticada simple

Considere el siguiente modelo simple conformado por un ensamblaje de 3 elementos. (Los archivos de datos de entrada estan disponibles en la carpeta **files** del REPO.)

<center><img src="images/frame2.png" alt="files" style="width:500px"></center>

Se requiere determinar el desplazaiento lateral de la estructra cuando es sometida a la carga laterla $P.$

<div class="alert alert-warning">

Encuentre la matriz de transformación bajo rotación $\lambda$ requerida para la formulación de la matriz de rigidez en el sistema de referencia global.

</div>

Las modificacions que se deben aplicar al código de resortes (o al de cerchas) solo estan relacionadas con el hecho de que ahora hay 3 grados de libertad en cada nodo.

In [18]:
%matplotlib inline        
import matplotlib.pyplot as plt
import numpy as np
import sympy as sym

Lea los archivos de entrada de la carpeta `files`.

In [19]:
def readin():
    nodes    = np.loadtxt('files/' + 'Fnodes.txt', ndmin=2)
    mats     = np.loadtxt('files/' + 'Fmater.txt', ndmin=2)
    elements = np.loadtxt('files/' + 'Feles.txt' , ndmin=2)
    loads    = np.loadtxt('files/' + 'Floads.txt', ndmin=2)

    return nodes, mats, elements, loads

La subrutina `eqcounter` cuenta ecuaciones y genera el arreglo de condiciones de frontera.

In [20]:
def eqcounter(nodes):
    nnodes = nodes.shape[0]
    IBC = np.zeros([nnodes, 3], dtype=np.integer)
    for i in range(nnodes):
        for k in range(3):
            IBC[i , k] = int(nodes[i , k+3])
    neq = 0
    for i in range(nnodes):
        for j in range(3):
            if IBC[i, j] == 0:
                IBC[i, j] = neq
                neq = neq + 1

    return neq, IBC

Ahora la función `DME` calcula la matriz de ensamblaje de ecuaciones.

In [21]:
def DME(nodes, elements):

    nels = elements.shape[0]
    IELCON = np.zeros([nels, 2], dtype=np.integer)
    DME    = np.zeros([nels, 6], dtype=np.integer)
    neq , IBC = eqcounter(nodes)
    ndof   = 6
    nnodes = 2
    for i in range(nels):
        for j in range(nnodes):
            IELCON[i, j] = elements[i, j+3]
            kk = IELCON[i, j]
            for l in range(3):
                DME[i, 3*j+l] = IBC[kk, l]
    return DME , IBC , neq

La función assembly usa el modelo y la matriz DME para calcular la matriz de rigidez global.


In [22]:
def assembly(elements, mats, nodes, neq, DME, uel=None):

    IELCON = np.zeros([2], dtype=np.integer)
    KG = np.zeros((neq, neq))
    nels = elements.shape[0]
    nnodes = 2
    ndof = 6
    for el in range(nels):
        elcoor = np.zeros([nnodes, 2])
        im     = np.int(elements[el , 2])
        par0= mats[im ,0]
        par1= mats[im , 1]
        for j in range(nnodes):
            IELCON[j] = elements[el , j+3]
            elcoor[j , 0] = nodes[IELCON[j], 1]
            elcoor[j , 1] = nodes[IELCON[j], 2]
        kloc = uelbeam2DU(elcoor, par0, par1)
        dme = DME[el , :ndof]
        for row in range(ndof):
            glob_row = dme[row]
            if glob_row != -1:
                for col in range(ndof):
                    glob_col = dme[col]
                    if glob_col != -1:
                        KG[glob_row, glob_col] = KG[glob_row, glob_col] +\
                                                 kloc[row, col]

    return KG

La rutina **uelbeam2D** usa las coordenadas de los nudos y los parámetros de material para calcular la matriz de rigidez local ya transformada al sistema de referencia global.


<div class="alert alert-warning">
    
Agregue un comentario a cada línea relevante de los códigos de las siguientes rutinas y úselos para escribir los pseudocódigos correspondientes. En particular identifique el calculo de la matriz de transformación bajo rotación $\lambda$.

</div>

In [23]:
def uelbeam2DU(coord, I , Emod):
    """2D-2-noded beam element
       without axial deformation

    Parametros
    ----------
    coord : ndarray
      Cordenadas nodales (2, 2).
    A : float
      Area de la seccion transversal.
    Emod : float
      Modulo de elasticidad (>0).

    Returna
    -------
    kl : ndarray
      Matriz de rigidez local para el elemento (4, 4).

    """
    vec = coord[1, :] - coord[0, :]
    nx = vec[0]/np.linalg.norm(vec)
    ny = vec[1]/np.linalg.norm(vec)
    L = np.linalg.norm(vec)
    Q = np.array([
        [-ny , nx ,   0 ,  0 ,  0 , 0 ],
        [  0 ,  0 , 1.0 ,  0 ,  0 , 0 ],
        [  0 ,  0 ,   0 ,-ny , nx , 0 ],
        [  0 ,  0  ,  0 ,  0 ,  0 , 1.0 ]])
    kl =(I*Emod/(L*L*L)) * np.array([
        [12.0, 6 , -12.0 , 6*L],
        [6,  4*L*L , -6*L , 2*L*L],
        [-12.0,  -6*L , 12.0 , -6*L],
        [6*L,  2*L*L , -6*L , 4*L*L]])
    kG = np.dot(np.dot(Q.T, kl), Q)
    return kG

La rutina `loadassem` forma el vector de cargas en los nudos.

In [24]:
def loadasem(loads, IBC, neq, nl):
 
    RHSG = np.zeros([neq])
    for i in range(nl):
        il = int(loads[i, 0])
        ilx = IBC[il , 0]
        ily = IBC[il , 1]
        ilT = IBC[il , 2]
        if ilx != -1:
            RHSG[ilx] = loads[i, 1]
        if ily != -1:
            RHSG[ily] = loads[i, 2]
        if ilT != -1:
            RHSG[ilT] = loads[i, 3]

    return RHSG

El programa principal mantiene la misma estructura que el programa de resortes, es decir, se tienen siguientes pasos:

* Lee el modelo.

* Determina la matriz de ensamblaje `DME`.

* Ensambla el sistema global de ecuaciones.

* Determina los desplazamientos globales `UG` tras resolver el sistema global.

In [25]:
nodes, mats, elements, loads = readin()
DME , IBC , neq = DME(nodes, elements)
KG   = assembly(elements, mats, nodes, neq, DME)
RHSG = loadasem(loads, IBC, neq, 1)
UG = np.linalg.solve(KG, RHSG)
print(UG)

[ 2.25000000e+01  2.00000000e+01  1.27557539e-16 -1.25918779e-15
  2.00000000e+01  4.88970566e-16]


<div class="alert alert-warning">
    
### Problemas propuestos

#### Problema 1

Implemente una función que calcule las fuerzas nodales en cada elemento y que verifique el equilibrio del sistema.

#### Problema 2

Determine la rigidez lateral de la estructura usando la relación:

$$k = \frac{P}{\delta}\, .$$

#### Problema 3

Repotencie la estructura de tal forma que la rigidez lateral se incremente por un factor de 2.0

#### Problema 4

Repare el portico mostrado en la figura adicionando elementos y/o imponiendo restricciones apropiadas a los desplazamientos. (Cree un nuevo paquete de archivos de datos).

#### Problema 5

Para el portico de la figura asuma que la conexión del nudo 5 es articulada e indique como esta condición cambia los resultados.

<center><img src="images/frame3.png" alt="files" style="width:500px"></center>

</div>

### References

Bathe, Klaus-Jürgen. (2006) Finite element procedures. Klaus-Jurgen Bathe. Prentice Hall International.

Juan Gómez, Nicolás Guarín-Zapata (2018). SolidsPy: 2D-Finite Element Analysis with Python, <https://github.com/AppliedMechanics-EAFIT/SolidsPy>.

In [26]:
# This bit of code is a class added to make the title nice  (thanks to @lorenABarba )
from IPython.core.display import HTML
def css_styling():
    styles = open('./styles/custom_barba.css', 'r').read()
    return HTML(styles)
css_styling()